<a href="https://colab.research.google.com/github/svvetal/ML_Projects/blob/main/Cats_vs_Dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"shubhamvetal","key":"41a9ef08ff4421d69ed8cac597fb274e"}'}

In [3]:
! mkdir ~/.kaggle

In [4]:
! cp kaggle.json ~/.kaggle/

In [5]:
! chmod 600  ~/.kaggle/kaggle.json

In [6]:
! kaggle datasets list

ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              237KB  2021-12-12 11:59:54          18960  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01          11560  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19           5936  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   2GB  2021-10-22 10:48:21           4026  
promptcloud/careerbuilder-job-listing-2020                  Careerbuilder Jo

In [7]:
!kaggle datasets download -d shaunthesheep/microsoft-catsvsdogs-dataset --force

 98% 769M/788M [00:07<00:00, 104MB/s]
100% 788M/788M [00:07<00:00, 108MB/s]


In [8]:
import zipfile

# Unzipping the Dataset
local_zip = './microsoft-catsvsdogs-dataset.zip'
zip_ref = zipfile.ZipFile(local_zip,'r')
zip_ref.extractall('./microsoft-catsvsdogs-dataset')
zip_ref.close()

In [11]:
import os
import tensorflow as tf
def CorruptImage():
    num_corrupted = 0
    for category in ("Cat", "Dog"):
        path = os.path.join("./microsoft-catsvsdogs-dataset/PetImages", category)

        for img in os.listdir(path):
            image_path = os.path.join(path, img)
            try:
                file_Obj = open(image_path, "rb")
                is_jfif = tf.compat.as_bytes("JFIF") in file_Obj.peek(10)
            finally:
                file_Obj.close()
            
            if not is_jfif:
                num_corrupted +=1
                os.remove(image_path)
    print("Total deleted images is = %d" % num_corrupted)

CorruptImage()

Total deleted images is = 1590


In [12]:
def NonJpg():
    j=0
    work_dir = "./microsoft-catsvsdogs-dataset/PetImages"
    for catagory in os.listdir(work_dir):
        sub_folder = os.path.join(work_dir, catagory)
        for name in os.listdir(sub_folder):
            img_path = os.path.join(sub_folder,name)
            if name.split('.')[1] == 'jpg':
                if os.path.getsize(img_path) <= 0:
                    if os.path.isfile(img_path) == False:
                        print(img_path)
                    os.remove(img_path)
                elif os.path.getsize(img_path) > 0:
                    j +=1
            if name.split('.')[1] != 'jpg':
                print(img_path)
                os.remove(img_path)
    print(j)
NonJpg()

./microsoft-catsvsdogs-dataset/PetImages/Cat/Thumbs.db
./microsoft-catsvsdogs-dataset/PetImages/Dog/Thumbs.db
23410


In [13]:
import os

# Training Directory
train_cats_dir = os.path.join('./microsoft-catsvsdogs-dataset/PetImages/Cat')
train_dogs_dir = os.path.join('./microsoft-catsvsdogs-dataset/PetImages/Dog')


In [14]:
train_cat_names = os.listdir(train_cats_dir)
print(train_cat_names[:10])

train_dog_names = os.listdir(train_dogs_dir)
print(train_dog_names[:10])

['4169.jpg', '9489.jpg', '9327.jpg', '10851.jpg', '7254.jpg', '11613.jpg', '166.jpg', '588.jpg', '2060.jpg', '1169.jpg']
['4169.jpg', '9489.jpg', '9327.jpg', '10851.jpg', '7254.jpg', '11613.jpg', '166.jpg', '588.jpg', '2060.jpg', '1169.jpg']


In [15]:
print('Total Number of Cat Images : ', len(os.listdir(train_cats_dir)))
print('Total Number of Dog Images : ', len(os.listdir(train_dogs_dir)))

Total Number of Cat Images :  11741
Total Number of Dog Images :  11669


In [16]:
# MODEL BUILDING

In [76]:
import tensorflow as tf

In [77]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(128,128,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The Third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),  
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 
    tf.keras.layers.Dense(1, activation='sigmoid')                             
])

In [78]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_28 (Conv2D)          (None, 126, 126, 16)      448       
                                                                 
 max_pooling2d_28 (MaxPoolin  (None, 63, 63, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_29 (Conv2D)          (None, 61, 61, 32)        4640      
                                                                 
 max_pooling2d_29 (MaxPoolin  (None, 30, 30, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_30 (Conv2D)          (None, 28, 28, 64)        18496     
                                                                 
 max_pooling2d_30 (MaxPoolin  (None, 14, 14, 64)      

In [112]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate = 0.0001),
              metrics = ['accuracy'])

In [113]:
# Let's split image dataset into Train & validation
from tensorflow.keras import preprocessing
image_size = (128, 128)

train_dataset = preprocessing.image_dataset_from_directory(
    './microsoft-catsvsdogs-dataset/PetImages',
    image_size = image_size,
    validation_split= 0.2,
    subset="training",
    batch_size =16, 
    seed=1337,   
    shuffle = True
)

test_dataset = preprocessing.image_dataset_from_directory(
    './microsoft-catsvsdogs-dataset/PetImages',
    image_size = image_size,
    validation_split = 0.2,
    subset = "validation",
    batch_size = 16,   
    seed=1337, 
    shuffle= True
)

Found 23410 files belonging to 2 classes.
Using 18728 files for training.
Found 23410 files belonging to 2 classes.
Using 4682 files for validation.


In [114]:
history = model.fit(
    train_dataset,
    steps_per_epoch=10,
    epochs=15,
    verbose=1,
    validation_data = test_dataset
)

Epoch 1/15
10/10 [==============================] - 6s 535ms/step - loss: 0.4818 - accuracy: 0.8062 - val_loss: 0.6372 - val_accuracy: 0.6937
Epoch 2/15
10/10 [==============================] - 5s 528ms/step - loss: 0.4290 - accuracy: 0.8188 - val_loss: 0.6241 - val_accuracy: 0.6965
Epoch 3/15
10/10 [==============================] - 5s 527ms/step - loss: 0.4666 - accuracy: 0.8000 - val_loss: 0.6278 - val_accuracy: 0.7038
Epoch 4/15
10/10 [==============================] - 5s 528ms/step - loss: 0.3489 - accuracy: 0.8625 - val_loss: 0.6165 - val_accuracy: 0.7067
Epoch 5/15
10/10 [==============================] - 5s 522ms/step - loss: 0.3660 - accuracy: 0.8625 - val_loss: 0.6075 - val_accuracy: 0.7117
Epoch 6/15
10/10 [==============================] - 5s 543ms/step - loss: 0.3927 - accuracy: 0.8500 - val_loss: 0.6149 - val_accuracy: 0.7155
Epoch 7/15
10/10 [==============================] - 5s 526ms/step - loss: 0.4159 - accuracy: 0.8313 - val_loss: 0.5997 - val_accuracy: 0.7176
Epoch 